<a href="https://colab.research.google.com/github/SamarthJ03/Fake-news-detector/blob/main/Fake_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing required libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import  re
from textblob import TextBlob
from wordcloud import WordCloud
import keras


In [ ]:
df = pd.read_csv("/content/fake_or_real_news.csv")

Exploring the data

In [ ]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
df.drop(columns='Unnamed: 0',inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   6335 non-null   object
 1   text    6335 non-null   object
 2   label   6335 non-null   object
dtypes: object(3)
memory usage: 148.6+ KB


In [ ]:
df.isnull().sum()

title    0
text     0
label    0
dtype: int64

In [ ]:
df_t = df.copy()

Encoding the target

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder();
df_t['label'] = le.fit_transform(df_t['label'])

In [ ]:
le.inverse_transform([0,1])

array(['FAKE', 'REAL'], dtype=object)

In [ ]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   6335 non-null   object
 1   text    6335 non-null   object
 2   label   6335 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 148.6+ KB


In [ ]:
df_t.describe()

,label
count,6335.000000
mean,0.500552
std,0.500039
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [ ]:
y = df_t['label']
df_t = df_t.drop(columns='label')

In [ ]:
df_t.duplicated().sum()

29

In [ ]:
df_t['news'] = df_t['title']+ " " + df_t["text"]

In [ ]:
df_t['news'].duplicated().sum()

29

In [ ]:
df_t[df_t['news'].duplicated()]["news"]

1492    University of Missouri, please immediately fir...
1954    Loretta Lynch becomes first African-American w...
2336    Obamacare Enrollees Anxiously Await Supreme Co...
2856    Donald Trump is blatantly racist — and the med...
2952    Black Agenda Report for Week of Oct 31, 2016 N...
3081    The College Loan Bombshell Hidden in the Budge...
3292    This astonishing chart shows how moderate Repu...
3301    OnPolitics | 's politics blog Who has Trump ap...
3324    Senate GOP prepared to replace Obamacare subsi...
4140    We Republicans Lost On Gay Rights. That’s A Go...
4405    Paintings 'almost no one else seemed to be doi...
4885    An Open Letter to Black South African Police O...
4971    US faces calls to ‘walk away’ from Iran talks ...
5083    Fighting Ghost Fascists While Aiding Real Ones...
5105    Clinton clinches Democratic nomination – Sande...
5127    OnPolitics | 's politics blog Who has Trump ap...
5414    Do Democrats Want What Bernie Wants, Or Just W...
5417    Black 

In [ ]:
df_t

,title,text,news
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",You Can Smell Hillary’s Fear Daniel Greenfield...
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,Watch The Exact Moment Paul Ryan Committed Pol...
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,Kerry to go to Paris in gesture of sympathy U....
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",Bernie supporters on Twitter erupt in anger ag...
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,The Battle of New York: Why This Primary Matte...
...,...,...,...
6330,State Department says it can't find emails fro...,The State Department told the Republican Natio...,State Department says it can't find emails fro...
6331,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,Anti-Trump Protesters Are Tools of the Oligarc...
6333,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...","In Ethiopia, Obama seeks progress on peace, se..."


In [ ]:
x = df_t.drop(columns=["title","text"])

Removing stopwords and lemmatizing the data for preprocessing

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:

from nltk.corpus import stopwords

sw_list = stopwords.words('english')
x['news'] = x['news'].apply(lambda y: [item for item in y.split() if item not in sw_list]).apply(lambda y:" ".join(y))
import string
exclude = string.punctuation
def remove_punc1(text):
    return text.translate(str.maketrans('', '', exclude))
x['news'] = x['news'].apply(remove_punc1)

In [ ]:
x['news']

0       You Can Smell Hillary’s Fear Daniel Greenfield...
1       Watch The Exact Moment Paul Ryan Committed Pol...
2       Kerry go Paris gesture sympathy US Secretary S...
3       Bernie supporters Twitter erupt anger DNC We t...
4       The Battle New York Why This Primary Matters I...
                              ...                        
6330    State Department says cant find emails Clinton...
6331    The ‘P’ PBS Should Stand ‘Plutocratic’ ‘Pentag...
6332    AntiTrump Protesters Are Tools Oligarchy  Info...
6333    In Ethiopia Obama seeks progress peace securit...
6334    Jeb Bush Is Suddenly Attacking Trump Heres Why...
Name: news, Length: 6335, dtype: object

In [ ]:
from nltk.corpus import wordnet

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()
x['news'] = x['news'].apply(lambda x: wordnet_lemmatizer.lemmatize(x) ).apply(lambda x:"".join(x))


In [ ]:
x["news"]

0       You Can Smell Hillary’s Fear Daniel Greenfield...
1       Watch The Exact Moment Paul Ryan Committed Pol...
2       Kerry go Paris gesture sympathy US Secretary S...
3       Bernie supporters Twitter erupt anger DNC We t...
4       The Battle New York Why This Primary Matters I...
                              ...                        
6330    State Department says cant find emails Clinton...
6331    The ‘P’ PBS Should Stand ‘Plutocratic’ ‘Pentag...
6332    AntiTrump Protesters Are Tools Oligarchy  Info...
6333    In Ethiopia Obama seeks progress peace securit...
6334    Jeb Bush Is Suddenly Attacking Trump Heres Why...
Name: news, Length: 6335, dtype: object

Tokenizing the data to convert it into numerical sequences

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(oov_token='<nothing>')

In [ ]:
sentences = x["news"]

In [ ]:
tokenizer.fit_on_texts(sentences)

In [ ]:
len(tokenizer.word_index)

110958

In [ ]:
train_df = tokenizer.texts_to_sequences(sentences)


In [ ]:
maxi = 0;
for i in range(len(train_df)):
  maxi = max(len(train_df[i]),maxi)
train_df = pad_sequences(train_df, maxlen=maxi, padding='post')

In [ ]:
maxi

12183

Building and Training model

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(train_df,y,test_size=0.2)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
import tensorflow as tf

In [ ]:
from keras.layers import Embedding

In [ ]:
model = Sequential()
model.add(Embedding(111669,2,input_length=12183))
model.add(Bidirectional((LSTM(32,return_sequences=True))))
model.add(Bidirectional((LSTM(32,return_sequences=True))))
model.add(Bidirectional((LSTM(32,return_sequences=False))))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 12183, 2)          223338    
                                                                 
 bidirectional_9 (Bidirecti  (None, 12183, 64)         8960      
 onal)                                                           
                                                                 
 bidirectional_10 (Bidirect  (None, 12183, 64)         24832     
 ional)                                                          
                                                                 
 bidirectional_11 (Bidirect  (None, 64)                24832     
 ional)                                                          
                                                                 
 dense_9 (Dense)             (None, 64)                4160      
                                                      

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(x_train, y_train, epochs=15, validation_data=(x_test, y_test), callbacks=[early_stopping])


Epoch 1/15
159/159 [==============================] - 340s 2s/step - loss: 0.5557 - accuracy: 0.6427 - val_loss: 0.2534 - val_accuracy: 0.8942
Epoch 2/15
159/159 [==============================] - 326s 2s/step - loss: 0.1553 - accuracy: 0.9432 - val_loss: 0.2053 - val_accuracy: 0.9053
Epoch 3/15
159/159 [==============================] - 312s 2s/step - loss: 0.0343 - accuracy: 0.9886 - val_loss: 0.2962 - val_accuracy: 0.9021
Epoch 4/15
159/159 [==============================] - 321s 2s/step - loss: 0.0075 - accuracy: 0.9978 - val_loss: 0.3947 - val_accuracy: 0.9013
Epoch 5/15
159/159 [==============================] - ETA: 0s - loss: 0.0044 - accuracy: 0.9984

In [121]:
model.save('my_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
